In [1]:
import pandas as pd
from utils import transformers_bert_completions, load_splits, load_models
from utils_model_sampling import sample_across_models, hyperparameter_utils

from yyy_analysis import examples_figure
import numpy as np

import os
from os.path import join, exists

import childespy
%load_ext autotime
%load_ext line_profiler
%pdb

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

Automatic pdb calling has been turned ON
time: 3.38 ms (started: 2022-01-28 16:47:15 -08:00)


In [2]:
import configuration
config = configuration.Config()

time: 2.03 ms (started: 2022-01-28 16:47:15 -08:00)


In [3]:
#os.environ["CUDA_VISIBLE_DEVICES"]="" #hangs as of 1/26 if GPU is enabled on workstation

time: 8.28 ms (started: 2022-01-28 16:47:15 -08:00)


# Success Example

In [4]:
# Make this config regenerate controlled later

pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"', db_version = "2020.1").iloc[0]['id']

regenerate = False
this_path = join(config.prov_csv_dir, 'pvd_utt_glosses.csv')

if regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, type from utterance where corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")
    utt_glosses.to_csv(this_path, index=False)
else: 
    utt_glosses = pd.read_csv(this_path)

R[write to console]: Using supported database version: '2020.1'.



time: 2.13 s (started: 2022-01-28 16:47:15 -08:00)


## Load phonology and context

In [5]:
all_tokens_phono = load_splits.load_phono()

time: 7.74 s (started: 2022-01-28 16:47:18 -08:00)


In [6]:
success_idx = 16928243

all_tokens_phono.loc[all_tokens_phono.utterance_id == success_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]


,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
997717,I want to read,,,16928243,997717,310,42336
997718,I want to read,aə,aə,16928243,997718,310,42336
997719,I want to read,wɑn,wɑnt,16928243,997719,310,42336
997720,I want to read,də,tu,16928243,997720,310,42336
997721,I want to read,wid,ɹid,16928243,997721,310,42336
997722,I want to read,,,16928243,997722,310,42336


time: 42.7 ms (started: 2022-01-28 16:47:25 -08:00)


In [7]:
target_transcript_id = 42336 # Corresponds to the success_idx 

time: 378 µs (started: 2022-01-28 16:47:25 -08:00)


In [8]:
# Broader context of sentences
utt_glosses.loc[(utt_glosses.transcript_id == target_transcript_id) &
                (utt_glosses.utterance_order.isin(range(310-10, 310+2)))] # Note to self: changed the range here

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
162628,Jasmine,42336,16928069,300,Lily,MOT,declarative
162639,a ballet,42336,16928081,301,Lily,CHI,declarative
162656,is Jasmine a ballerina,42336,16928098,302,Lily,MOT,question
162670,yeah,42336,16928113,303,Lily,CHI,declarative
162686,oh I didn't know that,42336,16928129,304,Lily,MOT,declarative
162710,I ready now yyy,42336,16928154,305,Lily,CHI,declarative
162730,whoa,42336,16928174,306,Lily,MOT,declarative
162744,yyy,42336,16928189,307,Lily,CHI,declarative
162760,yyy yyy,42336,16928205,308,Lily,CHI,declarative
162779,you want mamma let's see,42336,16928225,309,Lily,MOT,declarative


time: 35.8 ms (started: 2022-01-28 16:47:25 -08:00)


## Estimate Prior and Posterior Probability

In [66]:
# Changed this notebook to use the +/- 20 context newly generated versions.

# Written to match load_models.query_model_title
default_args = {
    'split' : 'all',
    'dataset' : 'all', 
    'is_tags' : False,
    'context_num' : 20,
}

childes_all_title = load_models.query_model_title(model_type = 'childes', **default_args)
adult_all_title = load_models.query_model_title(model_type = 'adult', **default_args)
unigram_title = 'CHILDES Unigram'
flat_title = 'Flat Unigram'

time: 2.24 ms (started: 2022-01-28 17:26:22 -08:00)


In [148]:
import imp
imp.reload(examples_figure)
imp.reload(transformers_bert_completions)
imp.reload(sample_across_models)
imp.reload(load_models)

<module 'utils.load_models' from '/home/stephan/notebooks/nicole/child-directed-listening/utils/load_models.py'>

time: 13.1 ms (started: 2022-01-28 17:59:16 -08:00)


In [149]:
# CDL + Context +/- 20 is needed
# BERT + Context +/- 20 is needed
# Childes on train data.

# How to load properly with sample across models?
which_models = [
    ('all', 'all', False, 0, 'data_unigram'),
    ('all', 'all', False, 20, 'childes'),
    ('all', 'all', False, 20, 'adult'),
    ('all', 'all', False, 0, 'flat_unigram'),
]

time: 1.71 ms (started: 2022-01-28 17:59:18 -08:00)


In [150]:
raw_scores_across_models = examples_figure.get_scores_across_models(success_idx, which_models, True)

Running model CHILDES Unigram...
Computing WFST path lengths...
Processing lambda value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Running model CHILDES BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...
Processing lambda value 1 of 20
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running model Adult BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...
Processing lambda value 1 of 20
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
Running model Flat Unigram...
Computing WFST path lengths...
Processing lambda value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
time: 45.4 s (started: 2022-01-28 17:59:19 -08:00)


In [14]:
#%lprun -f wfst.get_wfst_distance_matrix raw_scores_across_models = examples_figure.get_scores_across_models(success_idx, which_models, True)

time: 763 µs (started: 2022-01-28 16:48:44 -08:00)


In [151]:
scores_across_models = pd.concat(raw_scores_across_models)

time: 28.1 ms (started: 2022-01-28 18:00:07 -08:00)


In [152]:
scores_across_models = scores_across_models.loc[scores_across_models.likelihood_type == 'levdist']

time: 4.75 ms (started: 2022-01-28 18:00:08 -08:00)


In [153]:
np.unique(scores_across_models.likelihood_type)

array(['levdist'], dtype=object)

time: 4.44 ms (started: 2022-01-28 18:00:09 -08:00)


In [154]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type','prior_rank','posterior_rank']]
success_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token,likelihood_type,prior_rank,posterior_rank
997721,"CHILDES BERT without tags, , +-20 utts context",see read watch hear eat go be me dad look,0.593986775150996 0.28323569047071767 0.018986...,see go play look read know watch write mommy do,0.4728042 0.0909064 0.031111874 0.022768175 0....,0.018506,read,levdist,4,1.0


time: 27.3 ms (started: 2022-01-28 18:00:11 -08:00)


In [155]:
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
print('CDL+Context Prior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CDL+Context Prior


'see (0.47) go (0.09) play (0.03) look (0.02) read (0.02) know (0.02) watch (0.02) write (0.01) mommy (0.01) do (0.01)'

time: 14.8 ms (started: 2022-01-28 18:00:14 -08:00)


In [156]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('CDL+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CDL+Context Posterior


'see (0.59) read (0.28) watch (0.02) hear (0.02) eat (0.01) go (0.01) be (0.01) me (0.0) dad (0.0) look (0.0)'

time: 19.3 ms (started: 2022-01-28 18:00:16 -08:00)


In [157]:
success_example = scores_across_models.loc[(scores_across_models.model == adult_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

print('BERT+Context Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

BERT+Context Prior


'read (0.49) see (0.28) play (0.04) know (0.04) look (0.02) go (0.01) watch (0.01) help (0.01) try (0.01) talk (0.01)'

time: 10.1 ms (started: 2022-01-28 18:00:18 -08:00)


In [158]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('BERT+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],4))+')' for i in range(len(words))]) 

BERT+Context Posterior


'read (0.9503) see (0.0441) watch (0.0016) hear (0.0007) know (0.0005) be (0.0003) look (0.0002) eat (0.0002) weed (0.0002) go (0.0002)'

time: 18.4 ms (started: 2022-01-28 18:00:19 -08:00)


In [159]:
success_example = scores_across_models.loc[(scores_across_models.model == unigram_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token', 'prior_rank', 'posterior_rank']]
#print(success_example)
print('CHILDES-1Gram Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1Gram Prior


'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

time: 27.7 ms (started: 2022-01-28 18:00:21 -08:00)


In [160]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('CHILDES-1gram Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1gram Posterior


'we (0.2) the (0.08) and (0.07) need (0.06) read (0.04) one (0.04) what (0.03) would (0.03) he (0.03) me (0.02)'

time: 18.3 ms (started: 2022-01-28 18:00:23 -08:00)


In [161]:
success_example = scores_across_models.loc[(scores_across_models.model == flat_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example
print('UniformPrior Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

UniformPrior Prior


'stalks (0.0) rarely (0.0) harbor (0.0) dust (0.0) josh (0.0) milk (0.0) leon (0.0) pitch (0.0) blake (0.0) girlfriend (0.0)'

time: 25 ms (started: 2022-01-28 18:00:25 -08:00)


In [162]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('UniformPrior Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

UniformPrior Posterior


'weed (0.18) deed (0.01) week (0.01) feed (0.01) reed (0.01) seed (0.01) weak (0.01) wheat (0.01) wood (0.01) weeds (0.01)'

time: 17.2 ms (started: 2022-01-28 18:00:27 -08:00)


# Failure Example

In [163]:
import imp
imp.reload(examples_figure)
imp.reload(transformers_bert_completions)
imp.reload(sample_across_models)
imp.reload(load_models)

<module 'utils.load_models' from '/home/stephan/notebooks/nicole/child-directed-listening/utils/load_models.py'>

time: 8.8 ms (started: 2022-01-28 18:00:30 -08:00)


In [164]:
yyy_idx = 16813515

raw_scores_across_models = examples_figure.get_scores_across_models(yyy_idx, which_models, False)
scores_across_models = pd.concat(raw_scores_across_models)

Running model CHILDES Unigram...
Computing WFST path lengths...
Processing lambda value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Running model CHILDES BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...
Processing lambda value 1 of 20
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running model Adult BERT without tags, , +-20 utts context...
Computing failure scores
Computing success scores
Computing WFST path lengths...
Processing lambda value 1 of 20
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
Running model Flat Unigram...
Computing WFST path lengths...
Processing lambda value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
Computing edit distances...
Getting the Levenshtein distance matrix
Processing beta value 1 of 20
If possible compare the bert_token_id in sample_across_models to the bert_token_id in one of the other scores sets from bert.
time: 42.5 s (started: 2022-01-28 18:00:31 -08:00)


In [165]:
all_tokens_phono.loc[all_tokens_phono.utterance_id == yyy_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]

,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
289678,you make your yyy,,,16813515,289678,112,42253
289679,you make your yyy,ju,ju,16813515,289679,112,42253
289680,you make your yyy,meək,meək,16813515,289680,112,42253
289681,you make your yyy,jɜ,jɑɹ,16813515,289681,112,42253
289682,you make your yyy,fɜt,*,16813515,289682,112,42253
289683,you make your yyy,,,16813515,289683,112,42253


time: 35.7 ms (started: 2022-01-28 18:01:17 -08:00)


In [166]:
yyy_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) & (scores_across_models.likelihood_type == 'levdist')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type','posterior_rank']]
yyy_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token,likelihood_type,posterior_rank
289682,"CHILDES BERT without tags, , +-20 utts context",own hat feet bed food foot mess cat tea lot,0.24153853581354678 0.1188293613752251 0.05656...,own house bed dinner mouth breakfast hand name...,0.20277847 0.07381966 0.038039044 0.03174672 0...,NaN,NaN,levdist,NaN


time: 27.8 ms (started: 2022-01-28 18:01:19 -08:00)


In [167]:
words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))])

'own (0.2) house (0.07) bed (0.04) dinner (0.03) mouth (0.02) breakfast (0.02) hand (0.01) name (0.01) mess (0.01) tea (0.01)'

time: 14.3 ms (started: 2022-01-28 18:01:22 -08:00)


In [168]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'own (0.24) hat (0.12) feet (0.06) bed (0.05) food (0.04) foot (0.03) mess (0.02) cat (0.02) tea (0.02) lot (0.01)'

time: 16.5 ms (started: 2022-01-28 18:01:23 -08:00)


In [169]:
yyy_example = scores_across_models.loc[(scores_across_models.model == adult_all_title ) & (scores_across_models.likelihood_type == 'levdist')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'own (0.25) choice (0.24) point (0.04) bed (0.03) call (0.03) guess (0.03) wish (0.02) choices (0.02) move (0.02) mistake (0.02)'

time: 19.2 ms (started: 2022-01-28 18:01:41 -08:00)


In [170]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'own (0.32) bet (0.2) bed (0.04) call (0.04) cut (0.04) guess (0.04) wish (0.03) shot (0.03) choice (0.03) move (0.03)'

time: 17.2 ms (started: 2022-01-28 18:01:43 -08:00)


In [171]:
yyy_example = scores_across_models.loc[(scores_across_models.model == unigram_title) & (scores_across_models.likelihood_type == 'levdist')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

time: 21.9 ms (started: 2022-01-28 18:01:48 -08:00)


In [172]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'it (0.14) that (0.1) what (0.06) not (0.04) get (0.03) got (0.03) put (0.03) fit (0.03) feet (0.02) for (0.02)'

time: 16.7 ms (started: 2022-01-28 18:01:50 -08:00)


In [173]:
# Get the surrounding context
utt_glosses.loc[(utt_glosses.transcript_id == 42253) &
                (utt_glosses.utterance_order.isin(range(112-3, 112+3)))]

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
50658,then we won't be able to put them back into th...,42253,16813459,109,Alex,MOT,declarative
50679,do you want ta put some beans in your eggs and...,42253,16813482,110,Alex,MOT,question
50697,no,42253,16813501,111,Alex,CHI,declarative
50710,you make your yyy,42253,16813515,112,Alex,CHI,declarative
50723,can I make one,42253,16813529,113,Alex,MOT,question
50745,no,42253,16813554,114,Alex,MOT,declarative


time: 41.6 ms (started: 2022-01-28 18:01:51 -08:00)
